## Q2: Egocentric

Write a github action that lets you commit only if README.md contains your name.

In [19]:
#Here I'm taking inspiration from the manual: https://docs.github.com/en/actions/writing-workflows/workflow-syntax-for-github-actions#jobs
def create_yaml_instruction_file():
    with open('../.github/workflows/check_readme_name.yml', 'w') as f:
        f.write('''
name: Check README at commit
run-name: ${{ github.actor }} running action is Massimo Girola in the README?
on: [push]
jobs:
  Check-README:
    runs-on: ubuntu-latest
    outputs:
      output1: ${{ steps.check-readme-step.outputs.output1 }}
    steps:
      - run: echo "🎉 The job was automatically triggered by a ${{ github.event_name }} event."
      - run: echo "🐧 This job is now running on a ${{ runner.os }} server hosted by GitHub!"
      - run: echo "🔎 The name of your branch is ${{ github.ref }} and your repository is ${{ github.repository }}."
      - name: Check out repository code
        uses: actions/checkout@v4
      - run: echo "💡 The ${{ github.repository }} repository has been cloned to the runner."
      - run: echo "🖥️ The workflow is now ready to test your code on the runner."
      - id: check-readme-step  
        run: |
          if grep "Massimo Girola" README.md; then
            echo "output1=Massimo Girola is in the README" >> "$GITHUB_OUTPUT"
          else
            echo "output1=Massimo Girola is not in the README" >> "$GITHUB_OUTPUT"
          fi
      - run: echo "🍏 This job's status is ${{ job.status }}." 
      
  Authorize-the-Commit:
    runs-on: ubuntu-latest
    needs: Check-README
    steps:
      - name: Check out repository code
        uses: actions/checkout@v4
        with:
            fetch-depth: 0 #Apparently I have to clone everything to be able to reset and push
      - name: Set up Git account #Since this runs remotely I need to set up the git account
        run: |
            git config --global user.name "Massimo Girola"
            git config --global user.email "m.girola2@campus.unimib.it" 
      - env: 
            OUTPUT1: ${{ needs.Check-README.outputs.output1 }}
        run: |
            echo "The output of the previous job was $OUTPUT1"
            if ! grep -q "Massimo Girola" README.md; then
                echo "Error: README.md does not contain 'Massimo Girola'."
                git reset --hard HEAD~1 #since I cannot run the action before the commit or the push, I need to reset the last commit if the condition is not met. The exercise asks to let commit only if the name is in the README but this is not possible using actions that can happen only after the push.
                git push --force
                exit 1
            else
                echo "'Massimo Girola' found in README.md. Proceeding with the commit."
            fi
    ''')

create_yaml_instruction_file()


Ok, now it works. This file is pushed but since README.md is not present and therefore does not contain my name it reverts the push and the latest updates are not pushed.

Now I add the README.md

In [21]:
with open('../README.md', 'a') as f:
    f.write("Author of these excercises:\n Massimo Girola\n")

Now the commit and the push are not reverted and everything works.

I can push everything left behind from when the README.md did not existed.

## Q4: You're wrong

Find a bug/typo in the material of this course, fix it, and send me a pull request.

### In L04 Q7 you have written:

A convolution is defined as:  

  \begin{equation}                                                                           
    (f \star g)(t) \equiv \int_{-\infty}^{\infty} f(\tau) g(t - \tau) d\tau                   
  \end{equation}                                                                             

  It is easy to compute this with FFTs, via the _convolution theorem_,                                                                        
  \begin{equation}                                         
    \mathcal{F}\{f \star g\} = \mathcal{F}\{f\} \, \mathcal{F}\{g\}                                          
  \end{equation}                                         
  That is: the Fourier transform of the convolution of $f$ and $g$ is simply
  the product of the individual transforms of $f$ and $g$.  This allows us
  to compute the convolution via multiplication in Fourier space and then take
  the inverse transform, $\mathcal{F}^{-1}\{\}$, to recover the convolution in real space:
  
  \begin{equation}
  f \star g = \mathcal{F}^{-1}\{ \mathcal{F}\{f\} \, \mathcal{F}\{g\}\}
  \end{equation}
  
A common use of a convolution is to smooth noisy data, for example by convolving noisy data with a Gaussian.  We'll do that here.

### While the correct syntax is:

A convolution is defined as:  

\begin{equation}  
  (f \star g)(t) \equiv \int_{-\infty}^{\infty} f(\tau) g(t - \tau) d\tau \tag{1}  
\end{equation}  

It is easy to compute this with FFTs, via the _convolution theorem_,  

\begin{equation}  
  \mathcal{F}\{f \star g\} = \mathcal{F}\{f\} \, \mathcal{F}\{g\} \tag{2}  
\end{equation}  

That is: the Fourier transform of the convolution of $f$ and $g$ is simply the product of the individual transforms of $f$ and $g$. This allows us to compute the convolution via multiplication in Fourier space and then take the inverse transform, $\mathcal{F}^{-1}\{\}$, to recover the convolution in real space:

\begin{equation}  
  f \star g = \mathcal{F}^{-1}\{ \mathcal{F}\{f\} \, \mathcal{F}\{g\}\} \tag{3}  
\end{equation}  

A common use of a convolution is to smooth noisy data, for example by convolving noisy data with a Gaussian. We'll do that here.


### Otherwise all the equations are numbered with lable (1) and equation (2) is not properly formatted

I edit the file /Users/lustrair/SynologyDrive/PhD/Corsi/ScientificComputingWithPython/scientificcomputing_bicocca_2023_MG-1/lectures/L04_scipy.ipynb

Now I add, commit and push to my fork

Before submitting the excercises I will open the pull request (I hope not to forget to do so)